# DB Optimizer Example

This notebook shows how to analyze the Snowflake Explain Plan file using Pandas.

In [1]:
# Import libraries
import sys
import pandas as pd
import numpy as np

sys.path.append('..')
from src import functions as fx


In [2]:
# Read json data into dataframes
global_stats, operations = fx.readExplainPlanFile('../data/sample_explain_plan.json')

# global_stats holds partition and byte values
global_stats

,partitionsTotal,partitionsAssigned,bytesAssigned
0,2192,2192,33834182144


In [3]:
# All operations are loaded in their own dataframe
operations

,id,operation,expressions,parent,objects,alias,partitionsAssigned,partitionsTotal,bytesAssigned
0,0,\n\t\t\t\t,"[REGION.R_NAME, NATION.N_NAME, PART.P_MFGR, PA...",NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sort,"[REGION.R_NAME ASC NULLS LAST, NATION.N_NAME A...",0.0,NaN,NaN,NaN,NaN,NaN
2,2,InnerJoin,[joinKey: (REGION.R_REGIONKEY = NATION.N_REGIO...,1.0,NaN,NaN,NaN,NaN,NaN
3,3,Filter,[REGION.R_NAME IN 'ASIA' IN 'AMERICA'],2.0,NaN,NaN,NaN,NaN,NaN
4,4,TableScan,"[R_REGIONKEY, R_NAME]",3.0,[NEW_DBOE.NEW_RETAIL.REGION],REGION,1.0,1.0,2.048000e+03
5,5,InnerJoin,[joinKey: (NATION.N_NATIONKEY = SUPPLIER.S_NAT...,2.0,NaN,NaN,NaN,NaN,NaN
6,6,Filter,[NOT(NATION.N_NAME = 'VIETNAM')],5.0,NaN,NaN,NaN,NaN,NaN
7,7,JoinFilter,[joinKey: (REGION.R_REGIONKEY = NATION.N_REGIO...,6.0,NaN,NaN,NaN,NaN,NaN
8,8,TableScan,"[N_NATIONKEY, N_NAME, N_REGIONKEY]",7.0,[NEW_DBOE.NEW_RETAIL.NATION],NATION,1.0,1.0,3.072000e+03
9,9,InnerJoin,[joinKey: (SUPPLIER.S_SUPPKEY = PARTSUPP.PS_SU...,5.0,NaN,NaN,NaN,NaN,NaN


In [4]:
# Count of operations
operations['operation'].value_counts()

TableScan     5
InnerJoin     4
Filter        3
JoinFilter    3
\n\t\t\t\t    1
Sort          1
Name: operation, dtype: int64

In [5]:
# Objects scanned
operations[operations['operation'] == 'TableScan'].explode('objects')['objects'].value_counts()

NEW_DBOE.NEW_RETAIL.REGION      1
NEW_DBOE.NEW_RETAIL.NATION      1
NEW_DBOE.NEW_RETAIL.SUPPLIER    1
NEW_DBOE.NEW_RETAIL.PART        1
NEW_DBOE.NEW_RETAIL.PARTSUPP    1
Name: objects, dtype: int64

In [6]:
filter_expressions = operations[operations['operation'] == 'Filter'].explode('expressions')
child_ops = pd.merge(filter_expressions, operations, how='left', left_on='id', right_on='parent', suffixes=['', '_child'])
grandchild_ops = pd.merge(child_ops, operations, how='left', left_on='id_child', right_on='parent', suffixes=['', '_grandchild'])

filter_ops = grandchild_ops[['id', 'operation', 'expressions', 'objects_child', 'objects_grandchild', 'expressions_child', 'expressions_grandchild']]
filter_ops = filter_ops.explode('objects_child') \
    .explode('objects_grandchild') \
    .explode('expressions_child') \
    .explode('expressions_grandchild')

filter_ops['table'] = filter_ops['objects_child'].combine_first(filter_ops['objects_grandchild'])

filter_ops['column'] = np.where(filter_ops['expressions_child'].str.contains('joinKey:'), 
                                    filter_ops['expressions_grandchild'], filter_ops['expressions_child'])

filter_ops['filter_type'] = np.where(filter_ops['expressions'].str.contains(' IN '), 'IN',
                            np.where(filter_ops['expressions'].str.contains('NOT\('), 'NOT',
                            np.where(filter_ops['expressions'].str.contains('CONTAINS\('), 'CONTAINS', 'NA')))

filter_ops[['table', 'column', 'filter_type']]

,table,column,filter_type
0,NEW_DBOE.NEW_RETAIL.REGION,R_REGIONKEY,IN
0,NEW_DBOE.NEW_RETAIL.REGION,R_NAME,IN
1,NEW_DBOE.NEW_RETAIL.NATION,N_NATIONKEY,NOT
1,NEW_DBOE.NEW_RETAIL.NATION,N_NAME,NOT
1,NEW_DBOE.NEW_RETAIL.NATION,N_REGIONKEY,NOT
2,NEW_DBOE.NEW_RETAIL.PART,P_PARTKEY,CONTAINS
2,NEW_DBOE.NEW_RETAIL.PART,P_MFGR,CONTAINS
2,NEW_DBOE.NEW_RETAIL.PART,P_BRAND,CONTAINS
2,NEW_DBOE.NEW_RETAIL.PART,P_TYPE,CONTAINS
2,NEW_DBOE.NEW_RETAIL.PART,P_SIZE,CONTAINS


In [7]:
operations[operations['operation'] == 'TableScan'].explode('objects').explode('expressions')[['objects', 'expressions']] \
    .groupby(['objects']).agg(
        columns=('expressions', set)
    )

,columns
objects,
NEW_DBOE.NEW_RETAIL.NATION,"{N_NAME, N_REGIONKEY, N_NATIONKEY}"
NEW_DBOE.NEW_RETAIL.PART,"{P_MFGR, P_SIZE, P_RETAILPRICE, P_BRAND, P_TYP..."
NEW_DBOE.NEW_RETAIL.PARTSUPP,"{PS_SUPPLYCOST, PS_SUPPKEY, PS_AVAILQTY, PS_PA..."
NEW_DBOE.NEW_RETAIL.REGION,"{R_NAME, R_REGIONKEY}"
NEW_DBOE.NEW_RETAIL.SUPPLIER,"{S_NATIONKEY, S_ACCTBAL, S_SUPPKEY}"


In [8]:
# Filter expressions
operations[operations['operation'] == 'Filter'].explode('expressions')['expressions'].value_counts()

REGION.R_NAME IN 'ASIA' IN 'AMERICA'    1
NOT(NATION.N_NAME = 'VIETNAM')          1
CONTAINS(PART.P_TYPE, 'COPPER')         1
Name: expressions, dtype: int64

In [9]:
# Object partitions
operations.explode('objects')[['objects', 'partitionsAssigned', 'partitionsTotal', 'bytesAssigned']].groupby('objects').agg(['max'])

,partitionsAssigned,partitionsTotal,bytesAssigned
,max,max,max
objects,,,
NEW_DBOE.NEW_RETAIL.NATION,1.0,1.0,3.072000e+03
NEW_DBOE.NEW_RETAIL.PART,321.0,321.0,3.732420e+09
NEW_DBOE.NEW_RETAIL.PARTSUPP,1807.0,1807.0,2.960336e+10
NEW_DBOE.NEW_RETAIL.REGION,1.0,1.0,2.048000e+03
NEW_DBOE.NEW_RETAIL.SUPPLIER,62.0,62.0,4.984013e+08


In [10]:
# Expressions used
operations.explode('expressions')['expressions'].value_counts()

joinKey: (NATION.N_NATIONKEY = SUPPLIER.S_NATIONKEY)                               2
joinKey: (SUPPLIER.S_SUPPKEY = PARTSUPP.PS_SUPPKEY)                                2
joinKey: (REGION.R_REGIONKEY = NATION.N_REGIONKEY)                                 2
REGION.R_NAME                                                                      1
N_NAME                                                                             1
S_SUPPKEY                                                                          1
S_NATIONKEY                                                                        1
S_ACCTBAL                                                                          1
joinKey: (PART.P_PARTKEY = PARTSUPP.PS_PARTKEY)                                    1
CONTAINS(PART.P_TYPE, 'COPPER')                                                    1
P_PARTKEY                                                                          1
P_MFGR                                                           